<table width="100%"><tr><td style="color:#bbbbbb;background-color:#ffffff;font-size:11px;font-style:italic;text-align:right;">This cell contains some macros. If there is a problem with displaying mathematical formulas, please run this cell to load these macros. </td></tr></table>
$ \newcommand{\bra}[1]{\langle #1|} $
$ \newcommand{\ket}[1]{|#1\rangle} $
$ \newcommand{\braket}[2]{\langle #1|#2\rangle} $
$ \newcommand{\dot}[2]{ #1 \cdot #2} $
$ \newcommand{\biginner}[2]{\left\langle #1,#2\right\rangle} $
$ \newcommand{\mymatrix}[2]{\left( \begin{array}{#1} #2\end{array} \right)} $
$ \newcommand{\myvector}[1]{\mymatrix{c}{#1}} $
$ \newcommand{\myrvector}[1]{\mymatrix{r}{#1}} $
$ \newcommand{\mypar}[1]{\left( #1 \right)} $
$ \newcommand{\mybigpar}[1]{ \Big( #1 \Big)} $
$ \newcommand{\sqrttwo}{\frac{1}{\sqrt{2}}} $
$ \newcommand{\dsqrttwo}{\dfrac{1}{\sqrt{2}}} $
$ \newcommand{\onehalf}{\frac{1}{2}} $
$ \newcommand{\donehalf}{\dfrac{1}{2}} $
$ \newcommand{\hadamard}{ \mymatrix{rr}{ \sqrttwo & \sqrttwo \\ \sqrttwo & -\sqrttwo }} $
$ \newcommand{\vzero}{\myvector{1\\0}} $
$ \newcommand{\vone}{\myvector{0\\1}} $
$ \newcommand{\vhadamardzero}{\myvector{ \sqrttwo \\  \sqrttwo } } $
$ \newcommand{\vhadamardone}{ \myrvector{ \sqrttwo \\ -\sqrttwo } } $
$ \newcommand{\myarray}[2]{ \begin{array}{#1}#2\end{array}} $
$ \newcommand{\X}{ \mymatrix{cc}{0 & 1 \\ 1 & 0}  } $
$ \newcommand{\Z}{ \mymatrix{rr}{1 & 0 \\ 0 & -1}  } $
$ \newcommand{\Htwo}{ \mymatrix{rrrr}{ \frac{1}{2} & \frac{1}{2} & \frac{1}{2} & \frac{1}{2} \\ \frac{1}{2} & -\frac{1}{2} & \frac{1}{2} & -\frac{1}{2} \\ \frac{1}{2} & \frac{1}{2} & -\frac{1}{2} & -\frac{1}{2} \\ \frac{1}{2} & -\frac{1}{2} & -\frac{1}{2} & \frac{1}{2} } } $
$ \newcommand{\CNOT}{ \mymatrix{cccc}{1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0} } $
$ \newcommand{\norm}[1]{ \left\lVert #1 \right\rVert } $
$ \newcommand{\pstate}[1]{ \lceil \mspace{-1mu} #1 \mspace{-1.5mu} \rfloor } $

#  Quantum Approximate Optimization Algorithm (QAOA)

In this notebook, we will introduce the Quantum Approximate Optimization Algorithm (QAOA) and see how to implement the algorithm for the Max-Cut problem.


## What is QAOA?

QAOA, first introduced by Farhi et al. [2], is a hybrid quantum-classical algorithm (using both classical and quantum computation) that helps find an approximate solution for combinatoral optimization problems. 


## What are Combinatorial Optimization Problems?

In a combinatorial optimization problem, the goal is that given some binary constraints $\alpha$ over bitstrings of length n, $\it{z}\in \{0,1\}^{n}$, we want to find the bitstring that maximizes the objective function
$$
\text{argmax}_{\it{z}}C (\it{z}) = \text{argmax}_{\it{z}} \sum_{\alpha =1}^{m} C (\it{z})
$$
where 
$$
C_{\alpha} (\it{z}) = \begin{cases}
1, & \text{if} \, \it{z} \, \text{satisfies the constraint} \, \alpha \\
0, & \text{if} \, \it{z} \, \text{does not satisfy the constraint} \,\alpha \\
\end{cases}
$$

are functions that takes in a bitstring and outputs $0$ or $1$, depending on whether it satisfies the constraint. 

There are various different examples such as the Traveling Salesman problem and Max-Cut problem. Here, we will focus on solving a Max-Cut problem with QAOA.


## Why QAOA?

In QNickel, we saw an example of solving the Max-Cut problem with Grover's algorithm, so the question is why use QAOA? The answer is although there has been great progress, we do not have fault-tolerant quantum computers yet. So in the meantime, we use a quantum-classical hybrid approach, so that we only use quantum computers for parts of the problem where cassical computers having difficulty computing.

## How does QAOA work?

QAOA is a hybrid algorithm. A general hybrid algorithm works as follows

<img src="../Hybrid.png" width="50%" align="center">

There is a classical processing unit (CPU) and quantum processing unit (QPU). The idea is

1) The CPU sends a parametrized circuit to the QPU with some initial guess of parameters.

2) The QPU will then run this circuit for a number of shots and compute some observable and sample it.

3) We feed the sample back to the CPU. 

4) The CPU can then further optimize parameters for the objective function.

You repeat this cycle until you get a convergence to a good choice of paramters that optimizes your objective function.

Now let us look at a more specific example, the Max-Cut problem with QAOA.

For QAOA, you start with the state $\ket{+}$ and the ansatz or parametrized circuit is made of a chain of 'Cost' (specific to the Max-Cut problem) and 'Mixer' Hamiltonians 

$$
\ket{\psi} = e^{-i\beta_{p}M}e^{-i\gamma_{p}C}\cdots e^{-i\beta_{1}M}e^{-i\gamma_{1}C}\ket{+}
$$
where
\begin{align}
\text{Cost (C)}&=\frac{1}{2}\sum_{i,j=1}^{n}\omega_{ij}\frac{1-Z_{i}Z_{j}}{2}\\
\text{Mixer (M)}&=\sum_{j=1}^{n}X_{j}
\end{align}

and the quantum circuit will look like the folllowing

<img src="../QAOAcircuit.png" width="80%" align="center">

Let us look at each component more closely.


### Initial State

We start with the state

$$
\ket{+} = \frac{1}{\sqrt{2^{n}}}\sum_{x=0}^{2^{n}-1}\ket{x}
$$

which is just the equal superposition of all possible answers.

### Baker-Campbell-Hausdorff (BCH) Formula

Before moving on how we implement the two Hamiltonians in our quantum circuit, we first introduce the BCH formula, which is given by

$$
e^{X}e^{Y} = e^{Z} \quad \text{where} \quad Z = X+Y \quad \text{if} \quad [X,Y] =0
$$

\begin{align}
[ Z_{i}Z_{j},Z_{m}Z_{n}] &=0\\
e^{i\gamma\sum_{\langle ij\rangle}Z_{i}Z_{j}} &=  prod_{\langle ij\rangle} e^{Z}
\end{align}

\begin{align}
[ Z_{i}Z_{j},Z_{m}Z_{n} ]=0\\
\end{align}


### Cost Hamiltonian



### Mixer Hamiltonian

### References

[1] S. Alam, Rigetti Computing Lectures, IBA Karachi (2020).

[2] E. Farhi, J. Goldstone, and S. Gutmann, A Quantum Approximate Optimization Algorithm, arXiv.1411.4028 (2014).